In [1]:
import pandas as pd

In [2]:
# read the dataset
# I clean the dataset a little to reduced the process time
df = pd.read_csv('data_clean.csv') # expect this step to take about 30 seconds
df = df.drop(df.columns[0], axis=1) # unused index column
df.head()

,Content,Summary
0,New York police are concerned drones could bec...,Police have investigated criminals who have ri...
1,By . Ryan Lipman . Perhaps Australian porn sta...,Porn star Angela White secretly filmed sex act...
2,"This was, Sergio Garcia conceded, much like be...",American draws inspiration from fellow country...
3,An Ebola outbreak that began in Guinea four mo...,World Health Organisation: 635 infections and ...
4,By . Associated Press and Daily Mail Reporter ...,A sinkhole opened up at 5:15am this morning in...


In [3]:
# create BERT model
from summarizer import Summarizer
bert_model = Summarizer()

/Users/fengzheng/Desktop/Coding/Practice/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-11 21:24:20.012193: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.Lay

In [4]:
# get the first row
example = df.iloc[0]
example

Content    New York police are concerned drones could bec...
Summary    Police have investigated criminals who have ri...
Name: 0, dtype: object

In [5]:
# generate an summary
result = bert_model(example[0],max_length=200)
bert_summary = ''.join(result)
print(bert_summary)

New York police are concerned drones could become tools for terrorists, and are investigating ways to stop potential attacks. They are consulting with the military and members of its counterterrorism, bomb squad, emergency services and aviation units are working on a plan to counter weaponized drones. The NYPD now sees a drone carrying explosives as the number one threat.


/Users/fengzheng/Desktop/Coding/Practice/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [7]:
# cosine similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

X_list = word_tokenize(example[1])
Y_list = word_tokenize(bert_summary)
sw = stopwords.words('english') 
l1 =[];l2 =[]
  
# remove stop words from the string
X_set = {w for w in X_list if not w in sw} 
Y_set = {w for w in Y_list if not w in sw}
  
# form a set containing keywords of both strings 
rvector = X_set.union(Y_set) 
for w in rvector:
    if w in X_set: l1.append(1) # create a vector
    else: l1.append(0)
    if w in Y_set: l2.append(1)
    else: l2.append(0)
c = 0

In [8]:
# cosine formula 
for i in range(len(rvector)):
        c+= l1[i]*l2[i]
cosine = c / float((sum(l1)*sum(l2))**0.5)
print("similarity: ", cosine)

similarity:  0.25980762113533157
